### Parsing the annotation file

In [179]:
import csv
import json
import pandas as pd

In [2]:
with open('../docs/data/matrix_files/Pro-B_BM_EP.txt','r') as annot_file:
    lines = annot_file.readlines()

In [6]:
lines[0]

'chr1:13277140-13278940_ENSMUSG00000005886$Ncoa2$chr1$13364164$-\t2.565974\n'

In [7]:
lines[0].split("_")

['chr1:13277140-13278940',
 'ENSMUSG00000005886$Ncoa2$chr1$13364164$-\t2.565974\n']

In [12]:
lines[0].split("_")[0].split(":")[1].split("-")

['13277140', '13278940']

In [10]:
lines[0].split("_")[1].split("$")[1]

'Ncoa2'

In [23]:
chr_name = []
chr_start = []
chr_end = []
annot_name = []
for line in lines:
    chr_name.append(line.split("_")[0].split(":")[0])
    chr_start.append(line.split("_")[0].split(":")[1].split("-")[0])
    chr_end.append(line.split("_")[0].split(":")[1].split("-")[1])
    annot_name.append(line.split("_")[1].split("$")[1])

In [24]:
import pandas as pd

In [25]:
df = pd.DataFrame({
    "chr_name":chr_name,
    "chr_start":chr_start,
    "chr_end":chr_end,
    "annot_name":annot_name,
})

In [26]:
df.to_csv("annot_file.csv",index=False)

## Merging threshold interaction file with 3d structure

In [6]:
interaction_file = '../docs/data/human_inter.json'
structure_file = '../docs/data/dummy_chromatin_str.txt'

In [7]:
with open(interaction_file, 'r') as inter_file:
    inter_data = json.load(inter_file)

In [11]:
inter_data = inter_data['interactions']

In [113]:
with open(structure_file, 'r') as str_file:
    lines = str_file.readlines()

In [43]:
chr_str_pos_ = {
}
for i in range(22):
    chr_str_pos_[i+1] = []
    
    
for i in lines[1:]:
    chr_str_pos_[int(i.split("\t")[0])].append(i)

In [18]:
chr_pos = {
}
for i in range(22):
    chr_pos[i+1] = []

for i in inter_data:
    chr_pos[i['chr1']].append(i['pos1'])
    chr_pos[i['chr2']].append(i['pos2'])
    

In [25]:
for key,value in chr_pos.items():
    new_value = set(value)
    chr_pos[key] =new_value

In [53]:
chr_str_pos = {
}
for i in range(22):
    chr_str_pos[i+1] = []

for j in chr_str_pos_.values():
    for counter,i in enumerate(j):
        if counter in chr_pos[int(i.split("\t")[0])]:
            chr_str_pos[int(i.split("\t")[0])].append( str(counter) + "\t" +i)

In [54]:
with open('../docs/data/merge_3d_inter.txt','w') as outfile:
    for i in chr_str_pos.values():
        for j in i:
            outfile.write(j)

In [85]:
## Creating a final file for threejs

merged_file = '../docs/data/merge_3d_inter.txt'

with open(merged_file,'r') as mg_file:
    lines = mg_file.readlines()
    
    
points = []
    
for line in lines:
    dict_ = {
        
    }
    spl_line = line.split("\t")
    
    trans_intreract_list = []
    trans_intreract_list_freq = []
    trans_intreract_list_loc = []
    cis_intreract_list = []
    cis_intreract_list_freq = []
    cis_intreract_list_loc = []
    for i in inter_data:
        if i['pos1'] == int(spl_line[0]) and i['chr1'] != i['chr2']:
            trans_intreract_list.append(i['pos2'])
            trans_intreract_list_freq.append(i['freq'])
            for j in lines:
                if int(j.split("\t")[0]) == i['pos2']:
                    trans_intreract_list_loc.append([j.split("\t")[3],j.split("\t")[4],j.split("\t")[5][0:-2]])
                else:
                    continue
                
        elif i['pos1'] == int(spl_line[0]) and i['chr1'] == i['chr2']:
            cis_intreract_list.append(i['pos2'])
            cis_intreract_list_freq.append(i['freq'])
            for j in lines:
                if int(j.split("\t")[0]) == i['pos2']:
                    cis_intreract_list_loc.append([j.split("\t")[3], j.split("\t")[4], j.split("\t")[5][0:-2]])
                else:
                    continue
    
    dict_["trans_intreract_list"] = trans_intreract_list
    dict_["cis_intreract_list"] = cis_intreract_list 
    dict_["trans_intreract_list_freq"] = trans_intreract_list_freq
    dict_["cis_intreract_list_freq"] = cis_intreract_list_freq 
    dict_["trans_intreract_list_loc"] = trans_intreract_list_loc
    dict_["cis_intreract_list_loc"] = cis_intreract_list_loc 
    dict_["chr"] = spl_line[1]
    dict_["position"] = [spl_line[3],spl_line[4],spl_line[5][0:-2]]
    
    points.append(dict_)
            
    

In [86]:
data = {
    "data":points
}

In [88]:
with open('../docs/data/complete3d_network.json','w') as outfile:
    json.dump(data,outfile)

### Generating random annotation file for 3d structure

In [98]:
fixed_str_file = '../docs/data/dummy_chromatin_str_fixed.txt'

In [182]:
df = pd.read_csv("../docs/data/annot_file.csv")

annot_name = df['annot_name']
len(annot_name)

13344

In [127]:
with open(fixed_str_file , 'r') as fx_file:
    fx_lines = [line.split("\n")[0] for line in fx_file]
len(fx_lines)

13782

In [129]:
fx_lines[1].split("\t")

['1', '6831', '78.610', '57.461', '90.014']

In [130]:
import random

In [189]:
new_lines = []
for counter, j in enumerate(fx_lines[10:13344]):
    j = j.split("\t")
    line_ = {}
    line_['chr'] = j[0]
    
    x = random.choice([1, -1])
    y = random.randint(1,100000) * x
    
    z1 = float(j[1]) + y
    z2 = float(j[1]) - y
    
    z = [z1,z2]
    z.sort()

    line_['pos'] = [str(k) for k in z]
    line_['annot_name'] = annot_name[counter]
    line_['loc'] = ([j[2], j[3], j[4]])

    
    new_lines.append(line_)

In [190]:
data = {
    "annot_data":new_lines
}

with open('../docs/data/3d_str_annotation.json','w') as outfile:
    json.dump(data,outfile)